In [1]:
import os
path = 'C:\\workspaceWin\\ATEC\\code'
os.chdir(path)

In [2]:
import pandas as pd
from Levenshtein import distance
import jieba

In [3]:
jieba.load_userdict('../data/dict.txt')
df = pd.read_csv('../data/all.csv',encoding='utf-8-sig')
df.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\冯源\AppData\Local\Temp\jieba.cache
Loading model cost 0.631 seconds.
Prefix dict has been built succesfully.


,s1,s2,label
0,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1
1,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0
2,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0
3,如何得知关闭借呗,想永久关闭借呗,0
4,花呗扫码付钱,二维码扫描可以用花呗吗,0


In [4]:
a = jieba.lcut(df['s1'][0])
print(a)
a.remove('\ufeff')
print(a)

['\ufeff', '怎么', '更改', '花呗', '手机号码']
['怎么', '更改', '花呗', '手机号码']


In [5]:
df['len_s1'] = df.apply(lambda x: len(x['s1']),axis=1)
df['len_s2'] = df.apply(lambda x: len(x['s2']),axis=1)
df['len_s1s2_avg'] = (df['len_s1']+df['len_s2'])/2
df['len_s1_on_s2'] = df['len_s1']/df['len_s2']

df['len_diff'] = df.apply(lambda x: len(x['s1'])-len(x['s2']),axis=1)
df['edit_dst'] = df.apply(lambda x: distance(x['s1'],x['s2']),axis=1)

df['edit_dst_nzd'] = df['edit_dst']/df['len_s1s2_avg']

In [6]:
def get_counts(row):
    s1, s2 = row['s1'], row['s2']
    list_s1 = jieba.lcut(s1)
    list_s2 = jieba.lcut(s2)
    to_remove = ['， ','，','*','* ','!','! ','？','？ ','。','。 ','','\ufeff']
    list_s1_cnd = [x for x in list_s1 if x not in to_remove]
    list_s2_cnd = [x for x in list_s2 if x not in to_remove]
    set_s1 = set(list_s1_cnd)
    set_s2 = set(list_s2)
    return list_s1_cnd, list_s2_cnd, \
           len(list_s1_cnd), len(list_s2_cnd), \
           len(set_s1.intersection(set_s2)), \
           len(set_s1-set_s2), len(set_s2-set_s1) \


df[['l1_tkd','l2_tkd','len_l1_tkd','len_l2_tkd','common_wc','s1_uq_wc','s2_uq_wc']]=df.apply(get_counts, axis=1,result_type='expand')


In [7]:
def find_askwords(row):
    s1, s2 = row['l1_tkd'], row['l2_tkd']
    ask_words = ['如何','怎么','为什么','为何','咋么']
    s1_ask, s2_ask = False, False
    if len(s1)>0 and s1[0] in ask_words:
        s1_ask = True
    if len(s2)>0 and s2[0] in ask_words:
        s2_ask = True
    return s1_ask, s2_ask

df[['s1_ask','s2_ask']] = df.apply(find_askwords, axis=1,result_type='expand')
df.head()


,s1,s2,label,len_s1,len_s2,len_s1s2_avg,len_s1_on_s2,len_diff,edit_dst,edit_dst_nzd,l1_tkd,l2_tkd,len_l1_tkd,len_l2_tkd,common_wc,s1_uq_wc,s2_uq_wc,s1_ask,s2_ask
0,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1,11,30,20.5,0.366667,-19,24,1.170732,"[怎么, 更改, 花呗, 手机号码]","[我, 的, 花呗, 是, 以前, 的, 手机号码, 怎么, 更, 改成, 现在, 的, 支...",4,16,3,1,11,True,False
1,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0,15,10,12.5,1.500000,5,14,1.120000,"[也, 开, 不了, 花呗, 就, 这样, 了, 完事, 了]","[真的, 嘛, 就是, 花呗, 付款]",9,5,1,7,5,False,False
2,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0,11,13,12.0,0.846154,-2,11,0.916667,"[花呗, 冻结, 以后, 还, 能, 开通, 吗]","[我, 的, 条件, 可以, 开通, 花呗, 借款, 吗]",7,8,3,4,5,False,False
3,如何得知关闭借呗,想永久关闭借呗,0,8,7,7.5,1.142857,1,4,0.533333,"[如何, 得知, 关闭, 借呗]","[想, 永久, 关闭, 借呗]",4,4,2,2,2,True,False
4,花呗扫码付钱,二维码扫描可以用花呗吗,0,6,11,8.5,0.545455,-5,10,1.176471,"[花呗, 扫码, 付钱]","[二维码, 扫描, 可以, 用, 花呗, 吗]",3,6,1,2,5,False,False


In [8]:
df['len_l1l2_tkd_avg'] = (df['len_l1_tkd'] + df['len_l2_tkd']) / 2
df['common_wc_nzd'] = df['common_wc'] / (df['len_l1l2_tkd_avg']+1)
df['s1_uq_wc_nzd'] = df['s1_uq_wc']/(df['len_l1_tkd']+1)
df['s2_uq_wc_nzd'] = df['s2_uq_wc']/(df['len_l2_tkd']+1)


df.head()

,s1,s2,label,len_s1,len_s2,len_s1s2_avg,len_s1_on_s2,len_diff,edit_dst,edit_dst_nzd,...,len_l2_tkd,common_wc,s1_uq_wc,s2_uq_wc,s1_ask,s2_ask,len_l1l2_tkd_avg,common_wc_nzd,s1_uq_wc_nzd,s2_uq_wc_nzd
0,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1,11,30,20.5,0.366667,-19,24,1.170732,...,16,3,1,11,True,False,10.0,0.272727,0.2,0.647059
1,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0,15,10,12.5,1.500000,5,14,1.120000,...,5,1,7,5,False,False,7.0,0.125000,0.7,0.833333
2,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0,11,13,12.0,0.846154,-2,11,0.916667,...,8,3,4,5,False,False,7.5,0.352941,0.5,0.555556
3,如何得知关闭借呗,想永久关闭借呗,0,8,7,7.5,1.142857,1,4,0.533333,...,4,2,2,2,True,False,4.0,0.400000,0.4,0.400000
4,花呗扫码付钱,二维码扫描可以用花呗吗,0,6,11,8.5,0.545455,-5,10,1.176471,...,6,1,2,5,False,False,4.5,0.181818,0.5,0.714286


In [9]:
df['len_l2_tkd'].min()

0

In [10]:
from nltk.translate.bleu_score import sentence_bleu

def get_ngram_bleu(row):
    s1, s2 = row['s1'], row['s2']
    list_s1 = [[x for x in s1]]
    list_s2 = [x for x in s2]
    b1 = sentence_bleu(list_s1, list_s2, weights=(1, 0, 0, 0))
    b2 = sentence_bleu(list_s1, list_s2, weights=(0.5, 0.5, 0, 0))
    b3 = sentence_bleu(list_s1, list_s2, weights=(0.33, 0.33, 0.33, 0))
    b4 = sentence_bleu(list_s1, list_s2, weights=(0.25, 0.25, 0.25, 0.25))
    return b1, b2, b3, b4

df[['bleu1','bleu2','bleu3','bleu4']]=df.apply(get_ngram_bleu, axis=1,result_type='expand')

C:\Utilities\anaconda\envs\py36\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Utilities\anaconda\envs\py36\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Utilities\anaconda\envs\py36\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use Smoothi

In [11]:
df.shape

(102477, 27)

In [12]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
label,102477.0,0.182334,0.386121,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.000000
len_s1,102477.0,13.390956,6.111260,5.000000e+00,1.000000e+01,1.200000e+01,15.000000,97.000000
len_s2,102477.0,13.404520,6.134927,3.000000e+00,1.000000e+01,1.200000e+01,15.000000,112.000000
len_s1s2_avg,102477.0,13.397738,5.203689,5.000000e+00,1.000000e+01,1.250000e+01,15.000000,83.000000
len_s1_on_s2,102477.0,1.085702,0.467466,8.974359e-02,7.692308e-01,1.000000e+00,1.300000,7.000000
len_diff,102477.0,-0.013564,6.454159,-7.400000e+01,-3.000000e+00,0.000000e+00,3.000000,65.000000
edit_dst,102477.0,11.523649,6.212416,0.000000e+00,8.000000e+00,1.000000e+01,14.000000,91.000000
edit_dst_nzd,102477.0,0.838879,0.203561,0.000000e+00,7.058824e-01,8.571429e-01,1.000000,1.741176
len_l1_tkd,102477.0,7.590913,3.520895,2.000000e+00,5.000000e+00,7.000000e+00,9.000000,55.000000
len_l2_tkd,102477.0,7.595753,3.528848,0.000000e+00,5.000000e+00,7.000000e+00,9.000000,59.000000


In [13]:
df.to_csv('../data/trd_feated.csv',index=False)


